In [0]:
from pyspark.sql.types import *

schema = StructType([
    StructField("event_id", StringType()),
    StructField("timestamp", StringType()),
    StructField("service", StringType()),
    StructField("level", StringType()),
    StructField("message", StringType()),
    StructField("host", StringType()),
    StructField("response_time_ms", IntegerType())
])

bronze_df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .schema(schema)
    .load("/Volumes/real_time_projects/realtime_log_analytics_alerting/project_volume/data/kafka_logs/")
)

spark.sql("USE CATALOG real_time_projects")
spark.sql("USE SCHEMA realtime_log_analytics_alerting")

bronze_df.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/Volumes/real_time_projects/realtime_log_analytics_alerting/project_volume/checkpoints/bronze") \
    .outputMode("append") \
    .trigger(availableNow=True) \
    .table("bronze_logs")
